In [8]:
import numpy as np
from scipy.fft import fft, ifft


In [2]:
l_a = np.array([3/4, 1/2])
h_a = np.array([2/3, -2/3])

In [5]:
print(f'l+h {l_a+h_a}')
# para ser um filtro do tipo Haar a soma daria [1,0,0....] . Portanto não é um filtro de Haar

l+h [ 1.41666667 -0.16666667]


In [9]:
# Portanto é fazer o exemplo 7.3
# Primeiro passo é como converter os coeficientes do filtro em uma forma v_k e w_k
#v_k = (l_a * x)
#v_k = 3/4*x_k + 1/2*x_{k-1}

#w_k = (h_a * x)
#w_k = 2/3*(x_k - x_{k-1})

# foi só fazer a convolução da análise

array([[0.75, 0.5 ],
       [0.5 , 0.75]])

In [10]:
# Segundo passo
# U(D(l_a*x)) = v_k para k pares e 0 para k impares
# U(D(h_a*x)) = w_k para k pares e 0 para k impares

array([[0.5, 0.5],
       [0.5, 0.5]])

In [ ]:
# Terceiro passo
# entendi a equação antes da 7.7
# a 7.7 é apenas um reagrupamento menos óbvio
# colocamos em função de x_k e x_{k-1} porque nos próximos passos vamos gerar respostas a partir desses valores.


### Quarto passo

A equação anterior deve ser válida para qualquer x
aqui escolhemos um x específico para ver o que vai dar
escolhemos x = [1, 0,0, ...]. Só $x_0=1$ as outras posição ficam igual a zero.


consegui chegar a 4 equações
que teoricamente valem para qualquer n


In [11]:
def conv_circular(x, h, verbose=False):
    N = len(x)
    M = len(h)
    w = np.zeros(N)
    for k in range(N):
        if verbose: print(f'w[{k}] = ', end=' ')
        for m in range(M):
            if verbose: print(f'h[{m}] * x[{k-m}] ({h[m]}*{x[k-m]}={h[m] * x[k-m]})', end= '')
            if verbose and m+1 < M: print(' + ', end='')
            w[k] += h[m] * x[k-m]
        if verbose: print(f' = {w[k]}')
    return w

def test_conv_cicrular(N=100):
    x = np.random.rand(N)
    h = np.random.rand(5)
    
    golden = (ifft(fft(x,N) * fft(h,N)))
    dut = conv_circular(x,h)
    
    assert np.allclose(golden, dut), 'Teste falhou'
    print('Teste passou')
    
test_conv_cicrular()

Teste passou


In [15]:
x = np.array([1, -2, 2, 4])

In [16]:
def one_stage_analysis(x): 
    N = len(x)
    l_analysis = [3/4, 1/2]
    h_analysis = [2/3, -2/3]
    
    Xl = conv_circular(x,l_analysis)[::2]
    Xh = conv_circular(x,h_analysis)[::2]
    
    X = np.concatenate([Xl, Xh])
    return X, Xl, Xh

X, Xl, Xh = one_stage_analysis(x)
X

array([ 2.75      ,  0.5       , -2.        ,  2.66666667])

In [17]:
def upsample(x): return np.concatenate([np.insert(x, slice(1, None), 0), np.zeros(1)])

def one_stage_synthesis(X):
    N = len(X)
    # transformado em filtro causal 
    l_synthesis = [4/5, 4/5]
    h_synthesis = [-9/10, 3/5]
    
    Xl_condensed = X[:N//2]
    Xh_condensed = X[N//2:]
    Xl = upsample(Xl_condensed)
    Xh = upsample(Xh_condensed)
    
    vl = conv_circular(Xl, l_synthesis)
    vh = conv_circular(Xh, h_synthesis)
    
    x_delayed = vl + vh
    x = np.roll(x_delayed, -1)
    
    return x

one_stage_synthesis(X)

array([ 1., -2.,  2.,  4.])